#AmeniDC
## See the cost of amenities in the District of Columbia

In [1]:
import simplejson as json
import requests
from requests import Request, Session
from bs4 import BeautifulSoup
import re

from urlparse import urljoin
from collections import namedtuple
import sqlite3
from pyspark.sql import SQLContext

#import geojson
#from geojson import Feature, Point, FeatureCollection
import pprint
pp = pprint.PrettyPrinter(indent=0)

#### Read in all API keys

In [2]:
with open("../secrets/google_secrets.json.nogit") as fh:
    secrets = json.loads(fh.read())
key = secrets['server_api_key']

#### Establish connection to sqlite3 database

In [3]:
con = sqlite3.connect('./property_sale_points.db')

#OpenData.DC html scraping for metadata

In [101]:
def build_prop_metadata(attrs):

  url = 'https://www.arcgis.com/sharing/rest/content/items/'\
  '2acc75ccdd954267acecb8713b2b800a/info/metadata/metadata.xml?format=default&output=html'
  
  with requests.Session() as s:
    resp = s.get(url)

  if resp.status_code != 200:
    print 'Request not fulfilled. Aboring'
    return
  
  soup = BeautifulSoup(resp.text)

  con.execute("drop table if exists attributes")
  con.execute("create table attributes (code text, codedesc text)")

  tags = soup.find_all("em",text = re.compile('Attribute Label:'))
  code_desc = []
  for t in tags:
    dt_tag = t.find_parent("dl")
    if dt_tag:
      text_list = dt_tag.find_all(text=True)
      tmp_cd = [i for i in text_list if i!='\n' 
              and not re.search('Attribute Label:',i)
              and not re.search('Attribute Definition:',i) ]
      if len(tmp_cd) == 2:
        code_desc.append((tmp_cd[0],tmp_cd[1]))
      elif len(tmp_cd) == 1:
        pass
        code_desc.append((tmp_cd[0],''))
      else:
        pass

  # sort the attributes
  code_desc.sort()

  # con.rollback() is called after the with block finishes with an exception, the
  # exception is still raised and must be caught
  try:
    with con:
      con.execute("insert into attributes values (?,?)",('LAT_LNG','Latitude and longitude as string'))
      con.executemany("insert into attributes values (?,?)", 
                      (i for i in code_desc if i[0] in attrs))
  except sqlite3.IntegrityError:
      print "couldn't add values to the table"


In [102]:
attr_types = [('LAT_LNG', 'text'),
  ('ADDRESS1', 'text'),
  ('ADDRESS2', 'text'),
  ('ASSESSMENT', 'real'),
  ('CITYSTZIP', 'text'),
  ('HIGHNUMBER', 'real'),
  ('LOWNUMBER', 'real'),
  ('NBHD', 'real'),
  ('NEWIMPR', 'real'),
  ('NEWLAND', 'real'),
  ('NEWTOTAL', 'real'),
  ('OLDIMPR', 'real'),
  ('OLDLAND', 'real'),
  ('OLDTOTAL', 'real'),
  ('PROPTYPE', 'real'),
  ('QDRNTNAME', 'text'),
  ('SALEDATE', 'text'),
  ('SALEPRICE', 'text'),
  ('SUBNBHD', 'real'),
  ('UNITNUMBER', 'real'),
  ('USECODE', 'real'),
  ('VACLNDUSE', 'real')]

In [103]:
if 1:
  build_prop_metadata([a[0] for a in attr_types])

In [104]:
%%sql 

select * from attributes;

Done.


code,codedesc
LAT_LNG,Latitude and longitude as string
ADDRESS1,Tax billing address part 1
ADDRESS2,Tax billing address part 2
ASSESSMENT,Current Assessed Value Total: Total of land and building
CITYSTZIP,City State Zip
HIGHNUMBER,Ending House number of the property if it includes multiple addresses
LOWNUMBER,Base House Number of the property address
NBHD,Assessor's Neighborhood Code
NEWIMPR,Current appraised value of improvements such as buildings
NEWLAND,Current appraised land value


** Load sql magic for facilitated queries **

In [14]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [97]:
%%sql sqlite:///property_sale_points.db
  
SELECT * FROM attributes

Done.


code,codedesc
LAT/LNG,Latitude and longitude as string
ADDRESS1,Tax billing address part 1
ADDRESS2,Tax billing address part 2
ASSESSMENT,Current Assessed Value Total: Total of land and building
CITYSTZIP,City State Zip
HIGHNUMBER,Ending House number of the property if it includes multiple addresses
LOWNUMBER,Base House Number of the property address
NBHD,Assessor's Neighborhood Code
NEWIMPR,Current appraised value of improvements such as buildings
NEWLAND,Current appraised land value


#OpenData.DC API

Get the property sale points from http://opendata.dc.gov/datasets/2acc75ccdd954267acecb8713b2b800a_28
and store as a SQL database

In [120]:
def request_records():
  url = 'http://opendata.dc.gov/datasets/2acc75ccdd954267acecb8713b2b800a_28.geojson'

  with requests.Session() as s:
    resp = s.get(url)

  print resp.status_code
  if resp.status_code != 200:
    print 'Request not fulfilled. Aborting'  

  return resp.json()

def build_records(resp_jsonpes,attr_types):
  con.execute("drop table if exists records")
  qstring = 'create table records (' + ', '.join([i[0]+' '+i[1] for i in attr_types]) + ')'
  con.execute(qstring)
  istring = 'insert into records values (' + ','.join(['?' for i in xrange(len(attr_types))])+')'

  # con.rollback() is called after the with block finishes with an exception, the
  # exception is still raised and must be caught
  try:
    with con:
      for outer_dict in resp_json['features']:
        # Get all the property attributes
        row_dict = outer_dict['properties']
        coords = outer_dict['geometry']['coordinates']# Store the lat/lng as string
        row_dict.update({'LAT_LNG':str(coords[1])+','+str(coords[0])})
        con.execute(istring,tuple(row_dict[keep_key] for keep_key in [a[0] for a in attr_types]))

  except sqlite3.IntegrityError:
      print "couldn't add values to the table"

In [109]:
if 0:
  resp_json = request_records()

In [119]:
build_records(resp_json,attr_types)

38.9030487951,-77.0267201832
38.9030683973,-77.0267201905
38.9032920901,-77.026715845
38.9030568437,-77.0268117043
38.903076446,-77.0268117117
38.9030948951,-77.0268117186
38.9031698433,-77.0268132226
38.9034131385,-77.0268162665
38.9035688014,-77.0268148494
38.9035730336,-77.0267711338
38.9035319068,-77.0268030266
38.9032667029,-77.0268058782
38.9032228871,-77.0268058618
38.9032055911,-77.0268058553
38.9031629276,-77.0268058392
38.9034650538,-77.0266937677
38.9032898186,-77.0265638032
38.9033970525,-77.0265667946
38.9171092368,-77.0969366733
38.9170614852,-77.096935965
38.9170053919,-77.0968775341
38.9170888311,-77.09688022
38.9171170781,-77.0967764428
38.9352759632,-77.0590732312
38.9352596756,-77.0590726226
38.9352419911,-77.0590732042
38.9353938132,-77.0589953434
38.9354974791,-77.0590716271
38.927364757,-77.0275809991
38.9273402083,-77.0274058058
38.9273223997,-77.0274057989
38.9275616271,-77.0274296089
38.9274112317,-77.0274096399
38.9275606061,-77.0274999473
38.9275432932,-77.02

In [117]:
adict = {'a':1}
adict.update({'b':2})
adict

{'a': 1, 'b': 2}

In [69]:
result_to_csv = %sql select * from records
result_to_csv.csv('property_sales_points.csv')

Done.


CSV results at /home/vagrant/capstone/arnold-capstone/property_sales_points.csv

# OpenStreetMap API

Generate geospatial coordinates to query on the unit disk. Use openstreetmap API to reverse geocode coordinates.

base_lat = 38.904722
base_lng = -77.016389
rev_geocode_url = "http://nominatim.openstreetmap.org/reverse"
geocode_payload = {'format':'json','lat':base_lat,'lon':base_lng}
resp_address = requests.get(rev_geocode_url,params=geocode_payload)
resp_address.json()

In [63]:
%%sql
select count(proptype),avg(saleprice),* from records 
group by proptype


Done.


count(proptype),avg(saleprice),ADDRESS1,ADDRESS2,ASSESSMENT,CITYSTZIP,HIGHNUMBER,LOWNUMBER,NBHD,NEWIMPR,NEWLAND,NEWTOTAL,OLDIMPR,OLDLAND,OLDTOTAL,PROPTYPE,QDRNTNAME,SALEDATE,SALEPRICE,SUBNBHD,UNITNUMBER,USECODE,VACLNDUSE
706,13636638.5637,1700 K ST NW STE 600,,768770.0,"WASHINGTON, DC 20006-3816",,610.0,10.0,182280.0,597900.0,780180.0,172010.0,597900.0,769910.0,COMMERCIAL,NW,2013-01-02T00:00:00.000Z,1568125,,,59.0,N
1076,725010.142193,1 FINANCIAL PLZ,,1712380.0,"PROVIDENCE, RI 02903-2448",,622.0,39.0,1458150.0,372040.0,1830190.0,1333930.0,298240.0,1632170.0,FLATS/CONVERSIONS,NE,2012-07-24T00:00:00.000Z,3600000,J,,24.0,N
2019,2482899.17137,1224 R ST NW UNIT 1,,30000.0,"WASHINGTON, DC 20009-4350",,1210.0,40.0,21000.0,9000.0,30000.0,21000.0,9000.0,30000.0,GARAGE/UNIMPROVED LAND,NW,2012-11-19T00:00:00.000Z,745000,E,P-57C,18.0,N
18,41156504.2778,3 BETHESDA METRO CTR STE 1200,,89100000.0,"BETHESDA, MD 20814-6358",,2121.0,10.0,80863020.0,38300900.0,119163920.0,33199400.0,38300900.0,71500300.0,HOTELS/MOTELS,NW,2012-03-09T00:00:00.000Z,114536157,,,32.0,N
248,7874900.49597,7850 WALKER DR STE 400,,60387900.0,"GREENBELT, MD 20770-3240",,909.0,39.0,59939420.0,8606880.0,68546300.0,49659220.0,8606880.0,58266100.0,RESIDENTIAL-MULTI FAMILY,SE,2012-04-20T00:00:00.000Z,95427000,E,,22.0,N
14501,637625.764223,1210 R ST NW APT 316,,788170.0,"WASHINGTON, DC 20009-4877",,1210.0,40.0,597510.0,256080.0,853590.0,549500.0,235500.0,785000.0,RESIDENTIAL-SINGLE FAMILY,NW,2013-10-28T00:00:00.000Z,915000,E,316.0,17.0,N


# Google Maps Places API

Here, places are queried for nearby subway stations

In [66]:
def get_nearby_places(lat,lng,types):
  search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  # Add this here because key must go last
  search_url += '?location='+str(lat)+','+str(lng)
  search_payload = {"key":key,
                   "radius":1000,"types":types}
  req = Request('GET', search_url,params=search_payload)
  prepared = req.prepare()
  #print prepared.url

  with requests.Session() as s:
    resp = s.send(prepared)
  #print resp.status_code
  resp_json = resp.json()
  print json.dumps(resp_json,indent=4, separators=(',', ': '))
  

In [67]:
lat = 38.93
lng = -77.06
get_nearby_places(lat,lng,'cafe|grocery_or_supermarket')

{
    "status": "OK",
    "html_attributions": [],
    "results": [
        {
            "rating": 4.1,
            "name": "Open City",
            "reference": "CmRcAAAAsNczeTLnyA-4laH6WTirlXbknDgr7cIrLINpZPNMJhe50NOPyRbn8KJcix749lzuufKJr38Z1p9P0423CHEonqNAjdSWOkZwVbiNkAMpnR7Fv55w7-jXJG-MGCLRnGd2EhDzCmBo4ZqBCnIxXRjPzC4ZGhT-cmmywF_iUiWo2c3Jq2GuDVvdPQ",
            "price_level": 2,
            "geometry": {
                "location": {
                    "lat": 38.9237244,
                    "lng": -77.0523616
                }
            },
            "opening_hours": {
                "weekday_text": [],
                "open_now": true
            },
            "place_id": "ChIJFaK2ddS3t4kR4LLXUHpoLC0",
            "vicinity": "2331 Calvert Street Northwest, Washington",
            "photos": [
                {
                    "photo_reference": "CmRdAAAAVBm-VrjFwFaZwhbO_s4UVGi8M4fmRJlPCUBV5z1MP0vloQmtfhSeVsriXZRzDEXWjvQMFBuFP8OOyrsaAmTa_CFr4E9KvZ1_o4wSDeilSVBNUv8JikMJO

In [64]:
amenity_types = ['airport','cafe','grocery_or_supermarket','movie_theater','park',
                 'pharmacy','restaurant','shopping_mall','spa']

SyntaxError: invalid syntax (<ipython-input-64-597904257b20>, line 2)

#Google Distance Matrix API

Compute distances to subway stations

In [ ]:
# store results as list of dicts
results = resp_json['results'] 
Place = namedtuple('Place', ['place_id', 'name', 'lat','lng'], verbose=False)
stations = [Place(r['place_id'],r['name'],
                  r['geometry']['location']['lat'],
                  r['geometry']['location']['lng']) 
            for r in results]
for s in stations:
    print s.place_id, s.name, s.lat, s.lng


In [ ]:
search_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
# Add this here because key must go last
search_url += '?origins='+str(base_lat)+','+str(base_lng)
search_url += '&destinations='+str(stations[0].lat)+','+str(stations[0].lng)

print search_url


In [ ]:
search_payload = {"key":key}
req = Request('GET', search_url,params=search_payload)
prepared = req.prepare()
print prepared.url

session = Session()
resp = session.send(prepared)
resp_json = resp.json()
print resp.status_code
print json.dumps(resp_json,indent=4, separators=(',', ': '))